<a href="https://colab.research.google.com/github/joshuacalloway/csc575project/blob/main/new_toxic_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     |████████████████████████████████| 1.8MB 13.1MB/s 
     |████████████████████████████████| 3.2MB 46.8MB/s 
     |████████████████████████████████| 890kB 48.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=bf909edaef2009e73994e2281c23505de9c418ef3ab89aee9a082bb8ce07445e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!git clone https://github.com/NVIDIA/apex


Cloning into 'apex'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 7910 (delta 3), reused 13 (delta 1), pack-reused 7885
Receiving objects: 100% (7910/7910), 14.00 MiB | 3.85 MiB/s, done.
Resolving deltas: 100% (5388/5388), done.


In [4]:
%cd apex


/content/apex


In [ ]:
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-yebl_n66
Created temporary directory: /tmp/pip-req-tracker-6zmjq7kw
Created requirements tracker '/tmp/pip-req-tracker-6zmjq7kw'
Created temporary directory: /tmp/pip-install-_0l9xfgq
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-d0uuhf4n
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-6zmjq7kw'
    Running setup.py (path:/tmp/pip-req-build-d0uuhf4n/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.7.0+cu101


    running egg_info
    creating /tmp/pip-req-build-d0uuhf4n/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-d0uuhf4n/pip-egg-info/apex.egg-info/PK

In [ ]:
!pip install fast-bert

In [ ]:
!pip install torch

In [ ]:
from transformers import BertTokenizer
from pathlib import Path
import torch

from box import Box
import pandas as pd
import collections
import os
from tqdm import tqdm, trange
import sys
import random
import numpy as np
import apex
from sklearn.model_selection import train_test_split

import datetime

from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc

In [ ]:
torch.cuda.empty_cache()

In [ ]:
pd.set_option('display.max_colwidth', -1)
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

In [ ]:
DATA_PATH = Path('../data/')
LABEL_PATH = Path('../labels/')

AUG_DATA_PATH = Path('../data/data_augmentation/')

MODEL_PATH=Path('../models/')
LOG_PATH=Path('../logs/')
MODEL_PATH.mkdir(exist_ok=True)

model_state_dict = None

# BERT_PRETRAINED_PATH = Path('../../bert_models/pretrained-weights/cased_L-12_H-768_A-12/')
BERT_PRETRAINED_PATH = Path('../../bert_models/pretrained-weights/uncased_L-12_H-768_A-12/')
# BERT_PRETRAINED_PATH = Path('../../bert_fastai/pretrained-weights/uncased_L-24_H-1024_A-16/')
# FINETUNED_PATH = Path('../models/finetuned_model.bin')
FINETUNED_PATH = None
# model_state_dict = torch.load(FINETUNED_PATH)

LOG_PATH.mkdir(exist_ok=True)

OUTPUT_PATH = MODEL_PATH/'output'
OUTPUT_PATH.mkdir(exist_ok=True)

In [ ]:
args = Box({
    "run_text": "multilabel toxic comments with freezable layers",
    "train_size": -1,
    "val_size": -1,
    "log_path": LOG_PATH,
    "full_data_dir": DATA_PATH,
    "data_dir": DATA_PATH,
    "task_name": "toxic_classification_lib",
    "no_cuda": False,
    "bert_model": BERT_PRETRAINED_PATH,
    "output_dir": OUTPUT_PATH,
    "max_seq_length": 512,
    "do_train": True,
    "do_eval": True,
    "do_lower_case": True,
    "train_batch_size": 8,
    "eval_batch_size": 16,
    "learning_rate": 5e-5,
    "num_train_epochs": 6,
    "warmup_proportion": 0.0,
    "no_cuda": False,
    "local_rank": -1,
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "optimize_on_cpu": False,
    "fp16": True,
    "fp16_opt_level": "O1",
    "weight_decay": 0.0,
    "adam_epsilon": 1e-8,
    "max_grad_norm": 1.0,
    "max_steps": -1,
    "warmup_steps": 500,
    "logging_steps": 50,
    "eval_all_checkpoints": True,
    "overwrite_output_dir": True,
    "overwrite_cache": False,
    "seed": 42,
    "loss_scale": 128,
    "task_name": 'intent',
    "model_name": 'xlnet-base-cased',
    "model_type": 'xlnet'
})

In [ ]:
import logging

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

In [ ]:
logger.info(args)

In [ ]:
# tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_PATH, do_lower_case=args['do_lower_case'])

In [ ]:
device = torch.device('cuda')
if torch.cuda.device_count() > 1:
    args.multi_gpu = True
else:
    args.multi_gpu = False

In [ ]:
label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [ ]:
from fast_bert.prediction import BertClassificationPredictor

In [ ]:
predictor = BertClassificationPredictor(args.output_dir/'model_out', args.output_dir, LABEL_PATH, 
                                        multi_label=True, model_type='xlnet', do_lower_case=False)

In [ ]:
predictor = BertClassificationPredictor('../models/output/model_out', args.output_dir, LABEL_PATH, model_type='xlnet', do_lower_case=False)

In [ ]:
args.output_dir

In [ ]:
output = predictor.predict_batch(list(pd.read_csv("../data/test.csv")['comment_text'].values))

In [ ]:
pd.DataFrame(output).to_csv('../data/output_xlnet.csv')

In [ ]:
results = pd.read_csv('../data/output_xlnet.csv')

In [ ]:
preds = pd.DataFrame([{item[0]: item[1] for item in pred} for pred in output])

In [ ]:
preds.head()

In [ ]:
test_df = pd.read_csv("../data/train.csv")
test_df.head()

In [ ]:
output_df = pd.merge(test_df, preds, how='left', left_index=True, right_index=True)
del output_df['comment_text']

In [ ]:
columns = ['id','toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [ ]:
output_df = output_df[columns]

In [ ]:
output_df.to_csv('../data/output_xlnet.csv', index=None)

In [ ]:
pd.read_csv('../data/output_xlnet.csv', index_col='id')